In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

loaded_train_images = np.load('dataset/train_images_224.npy')
loaded_train_labels = pd.read_csv('dataset/train_labels_224.csv')
loaded_test_images = np.load('dataset/test_images_224.npy')
loaded_test_labels = pd.read_csv('dataset/test_labels_224.csv')

print(f"train images: {loaded_train_images.shape}")
print(f"train labels: {loaded_train_labels.shape}")
print(f"test images: {loaded_test_images.shape}")
print(f"test labels: {loaded_test_labels.shape}")

# sample images
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
np.random.seed(43)
random_indices = np.random.choice(len(loaded_train_images), size=8, replace=False)
for i in range(8):
    row = i // 4
    col = i % 4
    axes[row, col].imshow(loaded_train_images[random_indices[i]])
    label_info = loaded_train_labels.iloc[random_indices[i]]
    diseases = []
    if label_info['miner'] == 1:
        diseases.append('Miner')
    if label_info['rust'] == 1:
        diseases.append('Rust')
    if label_info['phoma'] == 1:
        diseases.append('Phoma')
    if not diseases:
        diseases.append('Healthy')
    axes[row, col].set_title(f"ID: {label_info['id']}\n{', '.join(diseases)}")
    axes[row, col].axis('off')

plt.tight_layout()
plt.show()